In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import math
import sklearn
import plotly 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential, save_model, load_model
from keras import Input 
from keras.layers import Dense, SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import plotly.express as px
import plotly.graph_objects as go
from pandas import read_csv
import matplotlib.pyplot as plt

## Setup

In [25]:

# Open all relevant files
f = open('radar_count_2022_09_01.csv') # Needs to be imported manually, not in the Git
df = pd.read_csv(f)
df=df.drop(['entity_id'], axis=1)
numberList = [0, 1, 2, 3, 4, 5]

# Randomly shuffle the list
df['randNumCol'] = random.choices(numberList, weights=(80, 10, 5, 3, 1, 1), k=330821)

# Attribute feature names
feature_names=['faixa', 'vehiclelight', 'speedlight', 'vehicleheavy', 'speedheavy', 'vehicleothers', 'speedothers', 'accelerationlight', 'accelerationheavy', 'accelerationothers']

X = df[feature_names]
Y = df['randNumCol']

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

# Create Decision Tree classifer object
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Linear Discriminant Analysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

# Print the accuracy of the model
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))
test=[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
print(lda.predict(np.array(test).reshape(1, -1))[0])


Accuracy of LDA classifier on training set: 0.80
Accuracy of LDA classifier on test set: 0.80
4


In [28]:
# Read from CSV
training_data = pd.read_csv('radar_count_2022_09_01.csv')
n = 330100
training_data.drop(training_data.tail(n).index,
        inplace = True)
training_data = training_data.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values

# Create MinMaxScaler object
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data.reshape(-1, 1))
x_training_data = []
y_training_data =[]

# Run through training data
for i in range(400, len(training_data)):

    x_training_data.append(training_data[i-400:i, 0])

    y_training_data.append(training_data[i, 0])

x_training_data = np.array(x_training_data)

y_training_data = np.array(y_training_data)

x_training_data = np.reshape(x_training_data, (x_training_data.shape[0], 

                                               x_training_data.shape[1], 

                                               1))


# Create the RNN
rnn = Sequential()
rnn.add(LSTM(units = 5, return_sequences = True, input_shape = (x_training_data.shape[1], 1)))
rnn.add(Dropout(0.2))
rnn.add(LSTM(units = 5, return_sequences = True))
rnn.add(Dropout(0.2))
rnn.add(LSTM(units = 5, return_sequences = True))
rnn.add(Dropout(0.2))
rnn.add(LSTM(units = 5))
rnn.add(Dropout(0.2))
rnn.add(Dense(units = 1))
rnn.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Train the RNN
rnn.fit(x_training_data, y_training_data, epochs = 1, batch_size = 4)

# Read from CSV
test_data = pd.read_csv('radar_count_2022_09_01.csv')
test_data = test_data.iloc[:, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
unscaled_training_data = pd.read_csv('radar_count_2022_09_01.csv')
unscaled_test_data = pd.read_csv('radar_count_2022_09_01.csv')

# Add all data to the test data
all_data = pd.concat((unscaled_training_data['randNumCo'], unscaled_test_data['randNumCo']), axis = 0)
x_test_data = all_data[len(all_data) - len(test_data) - 40:].values
x_test_data = np.reshape(x_test_data, (-1, 1))
x_test_data = scaler.transform(x_test_data)

final_x_test_data = []

# Run through new test data
for i in range(40, len(x_test_data)):

    final_x_test_data.append(x_test_data[i-40:i, 0])

final_x_test_data = np.array(final_x_test_data)

final_x_test_data = np.reshape(final_x_test_data, (final_x_test_data.shape[0], 

                                               final_x_test_data.shape[1], 

                                               1))

# Predict the new test data
predictions = rnn.predict(final_x_test_data)

1703/1703 [==============================] - 924s 519ms/step - loss: 0.0186


NameError: ignored